In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 💡 Romer Model of Endogenous Growth

Unlike the Solow model where long-run growth relies on *exogenous* technological progress, the Romer model provides a framework where technological advancement ($A$) is **endogenous**, meaning it is intentionally produced within the model.

The core idea is that **ideas (technology, knowledge)** are the engine of sustained long-run growth. New ideas are generated through research and development (R&D) activities, typically using labor ($L_A$) and the existing stock of ideas ($A$).

# ⚙️ Key Equations and Concepts

1.  **Idea Production Function:** Describes how new ideas ($\dot{A}$ or $\Delta A$) are created. A common simple form assumes the growth rate of ideas depends on the amount of labor devoted to R&D ($L_A$) and potentially the existing stock of ideas ($A$).
    $$ \dot{A} = \delta_A L_A A^{\phi} $$
    - $\delta_A$: Productivity of R&D workers.
    - $L_A$: Number of workers engaged in R&D.
    - $A$: Existing stock of ideas/knowledge.
    - $\phi$: Parameter capturing spillovers from existing ideas ("standing on shoulders" if $\phi > 0$, "fishing out" if $\phi < 0$).
    * **Simplified Version (used here):** Often simplified to $\dot{A} = \delta_A L_A A$ (i.e., $\phi=1$), which implies the *growth rate* $g_A = \dot{A}/A = \delta_A L_A$. *(Note: This version has strong scale effects, discussed below).*

2.  **Output Production Function:** Output ($Y$) is produced using ideas ($A$) and labor allocated to producing goods ($L_Y$). A simple form often used is linear in ideas:
    $$ Y = A L_Y $$
    *(Note: This differs from the Cobb-Douglas used in Solow/Hybrid models. It assumes ideas are non-rival and directly usable by all production workers).*

3.  **Labor Allocation:** The total labor force ($L$) grows at rate $n$ and is split between goods production ($L_Y$) and R&D ($L_A$) with a constant fraction $\omega$ in R&D:
    $$ L_A = \omega L $$
    $$ L_Y = (1 - \omega) L $$
    $$ L_{t+1} = L_t (1 + n) $$

**Non-Rivalry and Increasing Returns:** A key feature is that ideas ($A$) are **non-rival**: once created, an idea can be used by many people simultaneously without diminishing its usefulness. This leads to **increasing returns to scale** in the production of output with respect to *all* inputs (ideas $A$, capital $K$ if included, labor $L$). This increasing returns property is what allows for sustained per capita growth in the long run.

# 📈 Long-Run Growth and Scale Effects

In the simplified Romer model simulated here ($\dot{A}/A = \delta_A L_A = \delta_A \omega L$):

* **Growth Rate of Technology ($g_A$):**
    $$ g_A = \delta_A \omega L $$
    The growth rate of technology is proportional to the *total number* of researchers ($\omega L$).
* **Growth Rate of Output per Capita ($g_{Y/L}$):** Since $Y = A L_Y = A (1-\omega) L$, output per capita is $Y/L = A (1-\omega)$. Its growth rate is:
    $$ g_{Y/L} = g_A = \delta_A \omega L $$

**Scale Effect:** This simple model exhibits a strong **scale effect**: a larger total labor force ($L$) leads to a permanently higher growth rate of technology and output per capita. This prediction is controversial and doesn't fully match empirical evidence (e.g., large countries like China and India didn't always grow faster than smaller developed countries). More advanced Romer models (e.g., with $\phi < 1$ in the idea production function) can eliminate or modify this strong scale effect.

The simulation plots the levels of $A$, $Y$, and $Y/L$ (often on log scales to visualize growth rates) and calculates the implied long-run growth rate $g_A$.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def romer_endogenous_growth(z_delta_A=0.0005, L0=100.0, n=0.01, omega=0.1, A0=1.0, T=100):
    """
    Simulates a basic Romer model of endogenous growth.
    Y = A * L_Y
    A_dot/A = delta_A * L_A = delta_A * omega * L
    L grows at rate n.

    Args:
        z_delta_A (float): Productivity of R&D workers (delta_A in notes).
        L0 (float): Initial total labor force.
        n (float): Growth rate of the total labor force L.
        omega (float): Fraction of labor force in R&D (0 < omega < 1).
        A0 (float): Initial level of technology/ideas.
        T (int): Time Horizon (number of periods).
    """
    # Ensure T is an integer
    T = int(T)
    if T < 2: T = 2 # Need at least 2 periods for growth calc

    # Input validation
    omega = np.clip(omega, 0.01, 0.99)
    z_delta_A = max(z_delta_A, 0)
    n = max(n, 0)
    L0 = max(L0, 1)
    A0 = max(A0, 0.1)

    # Arrays to store results (size T+1 for levels)
    time = np.arange(T + 1)
    A = np.zeros(T + 1)
    L = np.zeros(T + 1)
    L_A = np.zeros(T + 1) # Labor in R&D
    L_Y = np.zeros(T + 1) # Labor in Goods Production
    Y = np.zeros(T + 1) # Total Output
    Y_per_L = np.zeros(T + 1) # Output per worker
    g_A = np.zeros(T)     # Growth rate of A (length T)
    g_Y_per_L = np.zeros(T) # Growth rate of Y/L

    # Initial conditions
    A[0] = A0
    L[0] = L0

    # --- Simulation Loop (t=0 to T-1 to calculate state for t+1) ---
    for t in range(T):
        # Labor allocation for period t
        L_A[t] = omega * L[t]
        L_Y[t] = (1 - omega) * L[t]

        # Production in period t
        Y[t] = A[t] * L_Y[t]
        Y_per_L[t] = A[t] * (1 - omega) # Y/L = A * L_Y / L

        # Calculate next period's state variables
        # Technology growth: A_dot/A = delta_A * L_A => A_t+1 = A_t * (1 + delta_A * L_A_t)
        current_g_A = z_delta_A * L_A[t]
        A[t+1] = A[t] * (1 + current_g_A)

        # Labor force growth
        L[t+1] = L[t] * (1 + n)

        # Store growth rates (from t to t+1)
        g_A[t] = current_g_A
        # Calculate Y[T] and Y_per_L[T] for the last period
        if t == T-1:
            L_A[T] = omega * L[T]
            L_Y[T] = (1 - omega) * L[T]
            Y[T] = A[T] * L_Y[T]
            Y_per_L[T] = A[T] * (1 - omega)
        # Growth rate of Y/L
        if Y_per_L[t] > 1e-9:
             g_Y_per_L[t] = (Y_per_L[t+1] / Y_per_L[t]) - 1
        else:
             g_Y_per_L[t] = 0 # Or NaN

    # --- Plotting ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.ravel()

    # Plot 0: Levels (Log Scale) - A, L
    axes[0].plot(time, A, label='Technology (A)', color='red', lw=2)
    axes[0].plot(time, L, label=f'Labor (L, n={n:.1%})', color='green', lw=2, linestyle=':')
    axes[0].set_yscale('log')
    axes[0].set_title('Technology & Labor Levels (Log Scale)')
    axes[0].set_xlabel("Time (t)")
    axes[0].set_ylabel("Level (Log Scale)")
    axes[0].legend(fontsize='small')
    axes[0].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 1: Output Levels (Log Scale) - Y, Y/L
    axes[1].plot(time, Y, label='Total Output (Y)', color='black', lw=2)
    axes[1].plot(time, Y_per_L, label='Output per Worker (Y/L)', color='purple', lw=2, linestyle='--')
    axes[1].set_yscale('log')
    axes[1].set_title('Output Levels (Log Scale)')
    axes[1].set_xlabel("Time (t)")
    axes[1].set_ylabel("Output (Log Scale)")
    axes[1].legend(fontsize='small')
    axes[1].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 2: Growth Rates - gA, g(Y/L)
    plot_time_g = time[:-1] # Time for growth rates (length T)
    axes[2].plot(plot_time_g, g_A*100, label='Tech. Growth (gA)', color='red', lw=2)
    axes[2].plot(plot_time_g, g_Y_per_L*100, label='Output per Worker Growth (g(Y/L))', color='purple', lw=2, linestyle='--')
    # Add line for n if n>0
    if n > 0:
        axes[2].axhline(n*100, color='green', linestyle=':', label=f'Labor Growth (n={n:.1%})')
    axes[2].set_title('Growth Rates')
    axes[2].set_xlabel("Time (t)")
    axes[2].set_ylabel("Growth Rate (%)")
    axes[2].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.2f}%'))
    axes[2].legend(fontsize='small')
    axes[2].grid(True, linestyle='--', alpha=0.6)

    # Plot 3: R&D Labor (LA)
    axes[3].plot(time, L_A, label=f'R&D Labor (L_A = omega*L)', color='blue', lw=2)
    axes[3].set_title('Labor Allocation to R&D')
    axes[3].set_xlabel("Time (t)")
    axes[3].set_ylabel("Number of R&D Workers (L_A)")
    axes[3].legend(fontsize='small')
    axes[3].grid(True, linestyle='--', alpha=0.6)
    axes[3].set_ylim(bottom=0)


    fig.suptitle(f"Romer Endogenous Growth Model (gA ≈ δA*ω*L)", fontsize=16, y=1.03)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    # --- Display Final Growth Rates & BGP Info ---
    final_gA = g_A[-1] if T > 0 else np.nan
    final_gYperL = g_Y_per_L[-1] if T > 0 else np.nan
    # BGP growth rate depends on whether L is growing (scale effect)
    bgp_gA_explanation = f"δA * ω * L({T}) = {z_delta_A:.4f} * {omega:.2f} * {L[-1]:.1f} ≈ {final_gA:.3%}" if T>0 else "N/A"
    bgp_gYperL_explanation = f"gA ≈ {final_gA:.3%}" if T>0 else "N/A"


    results_md = f"""
    ### 📈 Growth Results (Approximate Final Rates):

    * **Technology Growth (gA):** {final_gA:.3%}
    * **Output per Worker Growth (g(Y/L)):** {final_gYperL:.3%}
    * **Labor Growth (n):** {n:.3%}

    **Balanced Growth Path (BGP) Prediction:**
    * In this model, $g_A = \\delta_A L_A = \\delta_A \\omega L$.
    * If $n > 0$, $L$ grows, so $g_A$ increases over time (Scale Effect).
    * If $n = 0$, $L$ is constant, so $g_A$ should stabilize at $\\delta_A \\omega L_0$.
    * On the BGP, $g_{{Y/L}}$ should equal $g_A$.

    * **Final Calculated gA:** {bgp_gA_explanation}
    * **Final Calculated g(Y/L):** {bgp_gYperL_explanation}
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    romer_endogenous_growth,
    z_delta_A=FloatSlider(value=0.0005, min=0.0, max=0.005, step=0.0001, description='R&D Productivity (δA):', style=style, layout=layout, readout_format='.4f'),
    L0=FloatSlider(value=100.0, min=10, max=1000, step=10, description='Initial Labor (L0):', style=style, layout=layout, readout_format='.0f'),
    n=FloatSlider(value=0.01, min=0.0, max=0.05, step=0.002, description='Labor Growth (n):', style=style, layout=layout, readout_format='.1%'),
    omega=FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='R&D Share (omega ω):', style=style, layout=layout, readout_format='.2f'),
    A0=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Initial Tech (A0):', style=style, layout=layout, readout_format='.1f'),
    T=IntSlider(value=100, min=10, max=200, step=5, description='Time Periods (T):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=0.0005, description='R&D Productivity (δA):', layout=Layout(width='95%…

# 📘 Romer Model of Endogenous Growth

This model shows how **ideas (A)** grow over time from R&D investment:

\[
A_{t+1} = A_t + z A_t L_A, \quad Y_t = A_t L_Y
\]

Where:
- \( L_A = \omega L \): R&D labor
- \( L_Y = (1 - \omega)L \): production labor
- \( z \): productivity of R&D

🔑 Insight: Long-run growth comes from:
\[
g_A = z \cdot \omega L
\]

**More researchers → faster idea growth → faster economic growth**

**Sources**:  
- GrowthEcon [Ch. 7](https://growthecon.com/StudyGuide/romer.html)  
- Charles I. Jones, *Macroeconomics*, Ch. 8